In [ ]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing
import datetime as dt
from sklearn import model_selection
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:

# Load dataset 
train = pd.read_csv('asset/train.csv')
test = pd.read_csv('asset/test.csv')
sample = pd.read_csv('asset/sampleSubmission.csv')
weather = pd.read_csv('asset/weather.csv')
spray = pd.read_csv('asset/spray.csv')

# Get labels
labels = train.WnvPresent.values

# Creating dummy variables for the weather data
weather.CodeSum = weather.CodeSum.apply(str.split)
for (i,list_) in enumerate(weather.CodeSum):
    for item in list_:
        if item not in weather.columns:
            weather[item] = 0
        weather.set_value(col=item,index=i,value=1)


# Not using codesum anymore
weather = weather.drop('CodeSum', axis=1)
# Split station 1 and 2 and join horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)
weather.Date = pd.to_datetime(weather.Date)
spray.Date = pd.to_datetime(spray.Date)

In [ ]:
# for c in weather.columns:
#     print(c)
#     print (weather[c].value_counts())

In [ ]:
# replace some missing values and T with -1
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', 0)
weather = weather.replace(' T', 0)
weather = weather.replace('  T', 0)

In [ ]:
# Functions to extract month and day from dataset
train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day

# Add integer latitude/longitude columns
train['Lat_int'] = train.Latitude.apply(int)
train['Long_int'] = train.Longitude.apply(int)
test['Lat_int'] = test.Latitude.apply(int)
test['Long_int'] = test.Longitude.apply(int)

In [ ]:
# drop address columns
train = train.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos'], axis = 1)
test = test.drop(['Id', 'Address', 'AddressNumberAndStreet'], axis = 1)

In [ ]:
weather2 = weather.

In [7]:
# Merge with weather data
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')

# train = train.merge(spray, on='Date')
# test = test.merge(spray, on='Date')

In [20]:
# This function allows you to create columns for the data_df input that will have a value of 1 or zero
# If a trap has been sprayed with in the time period specified

def create_sprayed_cols(data_df, spray_df, time_period=2):
    # Iterating over unique dates that sprays took place
    # Sprays took place over 10 days as trucks drove around chicago
    for date in set(spray_df.Date):
        # I only want data for this unique date
        spray_temp = spray_df[spray_df.Date == date]
        # Resetting index to make iterating easier
        spray_temp.index = range(0, len(spray_temp))
        
        # I am creating a column for every unique date and initalizing it's rows to 0
        # I will set these values to 1 when I find a trap that was sprayed
        col_name = 'spray_'+date.strftime('%Y-%m-%d')+"_"+str(time_period)
        data_df[col_name] = 0

        # Iterating over each row of our training data to determine if a trap is in the location
        # of a spray. I am also checking to see if the spray was in the past
        for r in range(0,len(data_df)):
            if data_df.get_value(r,'Date') > date and data_df.get_value(r,'Date') < date + pd.Timedelta(weeks=time_period) :

                # I am casting the lat and long to ints, and multiplaying by 100 to truncate precision
                # In other words, I'm taking pin points and making them into squares
                cur_lat = int(data_df.get_value(r, 'Latitude') * 100)
                cur_long = int(data_df.get_value(r, 'Longitude') * 100)
                
                # Iterating over each value in my spray data
                for i in range(0, len(spray_temp)):

                    spray_lat = int(spray_temp.get_value(i,'Latitude')*100)
                    spray_long = int(spray_temp.get_value(i,'Longitude')*100)

                    # I am now checking if something is in the square +/- some threshold
                    if (cur_lat < spray_lat + 10 and cur_lat > spray_lat - 10) and \
                    (cur_long < spray_long + 10 and cur_long > spray_long - 10):
                        data_df.set_value(r,col_name, 1)
                        break

In [9]:
create_sprayed_cols(train, spray, time_period=10)
create_sprayed_cols(test,spray, time_period=10)

In [10]:
#Convert categorical data to numbers
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train['Species'].values) + list(test['Species'].values))
train['Species'] = lbl.transform(train['Species'].values)
test['Species'] = lbl.transform(test['Species'].values)

lbl.fit(list(train['Street'].values) + list(test['Street'].values))
train['Street'] = lbl.transform(train['Street'].values)
test['Street'] = lbl.transform(test['Street'].values)

lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train['Trap'] = lbl.transform(train['Trap'].values)
test['Trap'] = lbl.transform(test['Trap'].values)

train = pd.get_dummies(train, prefix = ['Trap', 'Species'],\
                                      columns=['Trap','Species'],drop_first=True)
test = pd.get_dummies(test, prefix = ['Trap', 'Species'],\
                                     columns=['Trap','Species'], drop_first=True)


In [11]:
# drop columns with -1s
train = train.loc[:,(train != -1).any(axis=0)]
test = test.loc[:,(test != -1).any(axis=0)]

In [12]:
train = train.drop('Date', axis=1)
test = test.drop('Date', axis=1)
train = train.drop(set(train.columns) - set(test.columns), axis=1)
test = test.drop(set(test.columns) - set(train.columns), axis=1)

In [13]:
_ = train.convert_objects(convert_numeric=True)
_ = test.convert_objects(convert_numeric=True)

/home/roland/anaconda3/envs/ga-immersive/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
/home/roland/anaconda3/envs/ga-immersive/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [14]:
def create_interactions(df):
    interaction = []
    for i in range(0,len(train.columns)-1):
        for j in range(i+1, len(train.columns)):
            interaction.append(train.columns[i] + '_x_' + train.columns[j])
    #This method returns a matrix of the origianl data, and the interaction of every column included
    data = preprocessing.PolynomialFeatures(2, include_bias=False, interaction_only=True).fit_transform(df)
    columns = list(df.columns) + interaction
    return pd.DataFrame(columns=columns, data=data)


In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Columns: 224 entries, Block to Species_6
dtypes: float64(4), int64(58), object(21), uint8(141)
memory usage: 95.2+ MB


In [16]:
# This didn't work that well, so I'm leaving it alone for now
# Don't accidently run this cell twice!!!!! You will blow up!
#train_int = create_interactions(train)
#test_int = create_interactions(test)

# Grid searching below

In [17]:
n_estimators = [300]
criterion = ['entropy']
max_depth = [2,5,10]
min_samples_split = [4,5]
max_features = [45,50]
clf = ensemble.GradientBoostingClassifier()

In [18]:
# Random Forest Classifier 
grid = model_selection.GridSearchCV(estimator=clf, scoring='roc_auc', param_grid=dict(n_estimators=n_estimators,
                                                    max_depth=max_depth, max_features=max_features, min_samples_split=min_samples_split))

grid.fit(train,labels)


GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_samples_split': [4, 5], 'max_features': [45, 50], 'n_estimators': [300], 'max_depth': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

### Evaluating results

In [19]:
grid.best_estimator_.fit(train, labels)

# create predictions and submission file
# predictions = grid.best_estimator_.predict_proba(test)[:,1]
# sample['WnvPresent'] = predictions
# sample.to_csv('beat_the_benchmark.csv', index=False)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=2,
              max_features=50, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=5, min_weight_fraction_leaf=0.0,
              n_estimators=300, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [21]:
grid.best_score_

0.58233200223410608

In [24]:
np.mean(model_selection.cross_val_score(grid.best_estimator_, X=train, y=labels, scoring='roc_auc', n_jobs=7))

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/roland/anaconda3/envs/ga-immersive/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/home/roland/anaconda3/envs/ga-immersive/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

..

In [25]:
model = grid.best_estimator_


#### Looking at feature importances

In [ ]:
# Zipping feature importances and sorting by how importnat they are
importances = zip(train.columns,model.feature_importances_)
importances.sort(key=lambda x: x[1])
importances = importances[::-1]
std = np.std([model.feature_importances_ for tree in model.estimators_],
             axis=0)

# Print the feature ranking
print("Feature ranking:")

for f in range(train.shape[1]):
    print("{}: {}".format(importances[f][0], importances[f][1]))

# Plot the feature importances of the forest
sns.barplot(x=train.columns, y=model.feature_importances_)


Feature ranking:
spray_2013-09-05_10_x_Trap_131: 0.0110722185925
DewPoint_y_x_Trap_14: 0.00740362665248
ResultDir_x_x_Trap_6: 0.0072790478255
Sunrise_x: 0.00583889425893
TSRA_y_x_Trap_107: 0.00562076511073
spray_2013-08-22_10_x_Trap_25: 0.00548465361124
spray_2013-08-15_10_x_Trap_140: 0.00507042961713
ResultSpeed_y_x_Trap_138: 0.00505892852186
FG_x_x_Trap_70: 0.00474226608729
ResultDir_x_x_Trap_88: 0.00470247315171
Latitude_x_FG_y: 0.0046908195772
FG_x_x_Trap_12: 0.00457757984681
WetBulb_x_x_Trap_52: 0.00447536073497
ResultDir_y_x_Trap_131: 0.00445942821627
TS_x_x_Trap_80: 0.00442614493008
AvgSpeed_y_x_Trap_138: 0.00434295255029
month_x_StnPressure_x: 0.00422845960218
SeaLevel_x_x_Trap_87: 0.00413069562297
PrecipTotal_x_x_Trap_6: 0.00408707195382
Latitude_x_ResultDir_y: 0.00404127876792
Depart_x_x_Trap_143: 0.00402300837471
spray_2013-08-22_10_x_Trap_30: 0.00400889094078
ResultSpeed_y_x_Trap_134: 0.00400116013762
ResultDir_x_x_Trap_4: 0.00392008755185
StnPressure_y_x_Trap_143: 0.003917

Trap_89_x_Trap_148: 0.0
Trap_89_x_Trap_147: 0.0
Trap_89_x_Trap_146: 0.0
Trap_89_x_Trap_145: 0.0
Trap_89_x_Trap_144: 0.0
Trap_89_x_Trap_143: 0.0
Trap_89_x_Trap_141: 0.0
Trap_89_x_Trap_140: 0.0
Trap_89_x_Trap_139: 0.0
Trap_89_x_Trap_138: 0.0
Trap_89_x_Trap_137: 0.0
Trap_89_x_Trap_136: 0.0
Trap_89_x_Trap_135: 0.0
Trap_89_x_Trap_134: 0.0
Trap_89_x_Trap_133: 0.0
Trap_89_x_Trap_132: 0.0
Trap_89_x_Trap_131: 0.0
Trap_89_x_Trap_130: 0.0
Trap_89_x_Trap_129: 0.0
Trap_89_x_Trap_128: 0.0
Trap_89_x_Trap_127: 0.0
Trap_89_x_Trap_123: 0.0
Trap_89_x_Trap_122: 0.0
Trap_89_x_Trap_121: 0.0
Trap_89_x_Trap_120: 0.0
Trap_89_x_Trap_119: 0.0
Trap_89_x_Trap_116: 0.0
Trap_89_x_Trap_115: 0.0
Trap_89_x_Trap_114: 0.0
Trap_89_x_Trap_113: 0.0
Trap_89_x_Trap_112: 0.0
Trap_89_x_Trap_111: 0.0
Trap_89_x_Trap_110: 0.0
Trap_89_x_Trap_109: 0.0
Trap_89_x_Trap_108: 0.0
Trap_89_x_Trap_107: 0.0
Trap_89_x_Trap_106: 0.0
Trap_89_x_Trap_105: 0.0
Trap_89_x_Trap_104: 0.0
Trap_89_x_Trap_103: 0.0
Trap_89_x_Trap_102: 0.0
Trap_89_x_Trap_1

Trap_34_x_Trap_43: 0.0
Trap_34_x_Trap_42: 0.0
Trap_34_x_Trap_41: 0.0
Trap_34_x_Trap_40: 0.0
Trap_34_x_Trap_39: 0.0
Trap_34_x_Trap_38: 0.0
Trap_34_x_Trap_37: 0.0
Trap_34_x_Trap_36: 0.0
Trap_34_x_Trap_35: 0.0
Trap_33_x_Species_6: 0.0
Trap_33_x_Species_5: 0.0
Trap_33_x_Species_4: 0.0
Trap_33_x_Species_3: 0.0
Trap_33_x_Species_2: 0.0
Trap_33_x_Species_1: 0.0
Trap_33_x_Trap_148: 0.0
Trap_33_x_Trap_147: 0.0
Trap_33_x_Trap_146: 0.0
Trap_33_x_Trap_145: 0.0
Trap_33_x_Trap_144: 0.0
Trap_33_x_Trap_143: 0.0
Trap_33_x_Trap_141: 0.0
Trap_33_x_Trap_140: 0.0
Trap_33_x_Trap_139: 0.0
Trap_33_x_Trap_138: 0.0
Trap_33_x_Trap_137: 0.0
Trap_33_x_Trap_136: 0.0
Trap_33_x_Trap_135: 0.0
Trap_33_x_Trap_134: 0.0
Trap_33_x_Trap_133: 0.0
Trap_33_x_Trap_132: 0.0
Trap_33_x_Trap_131: 0.0
Trap_33_x_Trap_130: 0.0
Trap_33_x_Trap_129: 0.0
Trap_33_x_Trap_128: 0.0
Trap_33_x_Trap_127: 0.0
Trap_33_x_Trap_123: 0.0
Trap_33_x_Trap_122: 0.0
Trap_33_x_Trap_121: 0.0
Trap_33_x_Trap_120: 0.0
Trap_33_x_Trap_119: 0.0
Trap_33_x_Trap_116:

Trap_7_x_Trap_119: 0.0
Trap_7_x_Trap_116: 0.0
Trap_7_x_Trap_115: 0.0
Trap_7_x_Trap_114: 0.0
Trap_7_x_Trap_113: 0.0
Trap_7_x_Trap_112: 0.0
Trap_7_x_Trap_111: 0.0
Trap_7_x_Trap_110: 0.0
Trap_7_x_Trap_109: 0.0
Trap_7_x_Trap_108: 0.0
Trap_7_x_Trap_107: 0.0
Trap_7_x_Trap_106: 0.0
Trap_7_x_Trap_105: 0.0
Trap_7_x_Trap_104: 0.0
Trap_7_x_Trap_103: 0.0
Trap_7_x_Trap_102: 0.0
Trap_7_x_Trap_101: 0.0
Trap_7_x_Trap_100: 0.0
Trap_7_x_Trap_99: 0.0
Trap_7_x_Trap_98: 0.0
Trap_7_x_Trap_97: 0.0
Trap_7_x_Trap_96: 0.0
Trap_7_x_Trap_95: 0.0
Trap_7_x_Trap_94: 0.0
Trap_7_x_Trap_93: 0.0
Trap_7_x_Trap_92: 0.0
Trap_7_x_Trap_91: 0.0
Trap_7_x_Trap_89: 0.0
Trap_7_x_Trap_88: 0.0
Trap_7_x_Trap_87: 0.0
Trap_7_x_Trap_86: 0.0
Trap_7_x_Trap_85: 0.0
Trap_7_x_Trap_84: 0.0
Trap_7_x_Trap_83: 0.0
Trap_7_x_Trap_82: 0.0
Trap_7_x_Trap_81: 0.0
Trap_7_x_Trap_80: 0.0
Trap_7_x_Trap_79: 0.0
Trap_7_x_Trap_78: 0.0
Trap_7_x_Trap_77: 0.0
Trap_7_x_Trap_76: 0.0
Trap_7_x_Trap_75: 0.0
Trap_7_x_Trap_71: 0.0
Trap_7_x_Trap_70: 0.0
Trap_7_x_Trap_

HZ_y_x_Trap_9: 0.0
HZ_y_x_Trap_8: 0.0
HZ_y_x_Trap_7: 0.0
HZ_y_x_Trap_6: 0.0
HZ_y_x_Trap_5: 0.0
HZ_y_x_Trap_4: 0.0
HZ_y_x_Trap_1: 0.0
HZ_y_x_spray_2013-08-22_10: 0.0
HZ_y_x_spray_2011-09-07_10: 0.0
HZ_y_x_spray_2013-08-08_10: 0.0
HZ_y_x_spray_2013-08-29_10: 0.0
HZ_y_x_spray_2013-07-25_10: 0.0
HZ_y_x_spray_2013-08-15_10: 0.0
HZ_y_x_spray_2013-09-05_10: 0.0
HZ_y_x_spray_2013-08-16_10: 0.0
HZ_y_x_spray_2011-08-29_10: 0.0
HZ_y_x_spray_2013-07-17_10: 0.0
HZ_y_x_GR_y: 0.0
HZ_y_x_VCFG_y: 0.0
HZ_y_x_SN_y: 0.0
HZ_y_x_SQ_y: 0.0
HZ_y_x_FG_y: 0.0
HZ_y_x_MIFG_y: 0.0
HZ_y_x_FG+_y: 0.0
HZ_y_x_BCFG_y: 0.0
HZ_y_x_DZ_y: 0.0
HZ_y_x_TS_y: 0.0
HZ_y_x_FU_y: 0.0
HZ_y_x_VCTS_y: 0.0
HZ_y_x_TSRA_y: 0.0
HZ_y_x_RA_y: 0.0
BR_y_x_Species_6: 0.0
BR_y_x_Species_5: 0.0
BR_y_x_Species_4: 0.0
BR_y_x_Species_3: 0.0
BR_y_x_Species_2: 0.0
BR_y_x_Species_1: 0.0
BR_y_x_Trap_148: 0.0
BR_y_x_Trap_147: 0.0
BR_y_x_Trap_146: 0.0
BR_y_x_Trap_145: 0.0
BR_y_x_Trap_144: 0.0
BR_y_x_Trap_143: 0.0
BR_y_x_Trap_141: 0.0
BR_y_x_Trap_140: 0.

FG+_x_x_Trap_112: 0.0
FG+_x_x_Trap_111: 0.0
FG+_x_x_Trap_110: 0.0
FG+_x_x_Trap_109: 0.0
FG+_x_x_Trap_108: 0.0
FG+_x_x_Trap_107: 0.0
FG+_x_x_Trap_106: 0.0
FG+_x_x_Trap_105: 0.0
FG+_x_x_Trap_104: 0.0
FG+_x_x_Trap_103: 0.0
FG+_x_x_Trap_102: 0.0
FG+_x_x_Trap_101: 0.0
FG+_x_x_Trap_100: 0.0
FG+_x_x_Trap_99: 0.0
FG+_x_x_Trap_98: 0.0
FG+_x_x_Trap_97: 0.0
FG+_x_x_Trap_96: 0.0
FG+_x_x_Trap_95: 0.0
FG+_x_x_Trap_94: 0.0
FG+_x_x_Trap_93: 0.0
FG+_x_x_Trap_92: 0.0
FG+_x_x_Trap_91: 0.0
FG+_x_x_Trap_89: 0.0
FG+_x_x_Trap_88: 0.0
FG+_x_x_Trap_87: 0.0
FG+_x_x_Trap_86: 0.0
FG+_x_x_Trap_85: 0.0
FG+_x_x_Trap_84: 0.0
FG+_x_x_Trap_83: 0.0
FG+_x_x_Trap_82: 0.0
FG+_x_x_Trap_81: 0.0
FG+_x_x_Trap_80: 0.0
FG+_x_x_Trap_79: 0.0
FG+_x_x_Trap_78: 0.0
FG+_x_x_Trap_77: 0.0
FG+_x_x_Trap_76: 0.0
FG+_x_x_Trap_75: 0.0
FG+_x_x_Trap_71: 0.0
FG+_x_x_Trap_70: 0.0
FG+_x_x_Trap_69: 0.0
FG+_x_x_Trap_68: 0.0
FG+_x_x_Trap_67: 0.0
FG+_x_x_Trap_66: 0.0
FG+_x_x_Trap_65: 0.0
FG+_x_x_Trap_64: 0.0
FG+_x_x_Trap_63: 0.0
FG+_x_x_Trap_62: 0.0


Heat_x_x_TS_y: 0.0
Heat_x_x_FU_y: 0.0
Heat_x_x_VCTS_y: 0.0
Heat_x_x_TSRA_y: 0.0
Heat_x_x_RA_y: 0.0
Heat_x_x_HZ_y: 0.0
Heat_x_x_BR_y: 0.0
Heat_x_x_AvgSpeed_y: 0.0
Heat_x_x_ResultDir_y: 0.0
Heat_x_x_ResultSpeed_y: 0.0
Heat_x_x_SeaLevel_y: 0.0
Heat_x_x_StnPressure_y: 0.0
Heat_x_x_PrecipTotal_y: 0.0
Heat_x_x_Cool_y: 0.0
Heat_x_x_Heat_y: 0.0
Heat_x_x_WetBulb_y: 0.0
Heat_x_x_DewPoint_y: 0.0
Heat_x_x_Tavg_y: 0.0
Heat_x_x_Tmin_y: 0.0
Heat_x_x_Tmax_y: 0.0
Heat_x_x_GR_x: 0.0
Heat_x_x_VCFG_x: 0.0
Heat_x_x_SN_x: 0.0
Heat_x_x_SQ_x: 0.0
Heat_x_x_FG_x: 0.0
Heat_x_x_MIFG_x: 0.0
Heat_x_x_FG+_x: 0.0
Heat_x_x_BCFG_x: 0.0
Heat_x_x_DZ_x: 0.0
Heat_x_x_TS_x: 0.0
Heat_x_x_FU_x: 0.0
Heat_x_x_VCTS_x: 0.0
Heat_x_x_TSRA_x: 0.0
Heat_x_x_RA_x: 0.0
Heat_x_x_HZ_x: 0.0
Heat_x_x_BR_x: 0.0
Heat_x_x_AvgSpeed_x: 0.0
Heat_x_x_ResultSpeed_x: 0.0
Heat_x_x_SeaLevel_x: 0.0
Heat_x_x_StnPressure_x: 0.0
Heat_x_x_PrecipTotal_x: 0.0
Heat_x_x_SnowFall_x: 0.0
Heat_x_x_Depth_x: 0.0
Heat_x_x_Sunset_x: 0.0
Heat_x_x_Sunrise_x: 0.0
Heat_x

KeyboardInterrupt: 

In [ ]:
grid.best_params_